<a href="https://colab.research.google.com/github/amitxsethi/Text2Sql-AIML/blob/main/gensql_group6_amit_tinyllama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Sun Nov 16 09:23:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             26W /   70W |   10836MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
!pip install nbstripout

The `nbstripout` tool can help clean up notebook metadata, which might resolve the rendering issue. You'll need to specify the exact filename of this notebook. After running the command, you might need to refresh your browser page for the changes to take effect.

In [14]:
!pwd
!nbstripout '../gensql-group6-amit-tinyllama.ipynb'
print("To apply nbstripout, uncomment the line above and replace 'YOUR_NOTEBOOK_NAME.ipynb' with the correct filename, then run this cell. Remember to refresh your page afterwards.")

/content
Could not strip '../gensql-group6-amit-tinyllama.ipynb': file not found
To apply nbstripout, uncomment the line above and replace 'YOUR_NOTEBOOK_NAME.ipynb' with the correct filename, then run this cell. Remember to refresh your page afterwards.


In [ ]:
!git clone https://github.com/jieshi10/gensql.git
%cd gensql
!ls -la
print("✓ Repository cloned!")

Cloning into 'gensql'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 149 (delta 29), reused 148 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (149/149), 2.42 MiB | 3.40 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/gensql/gensql
total 132
drwxr-xr-x 10 root root  4096 Nov 16 09:23 .
drwxr-xr-x 14 root root  4096 Nov 16 09:23 ..
drwxr-xr-x  4 root root  4096 Nov 16 09:23 benchmark
drwxr-xr-x  3 root root  4096 Nov 16 09:23 bird_code
drwxr-xr-x  2 root root  4096 Nov 16 09:23 docker
drwxr-xr-x  8 root root  4096 Nov 16 09:23 .git
-rw-r--r--  1 root root  3139 Nov 16 09:23 .gitignore
-rw-r--r--  1 root root  1064 Nov 16 09:23 LICENSE
-rw-r--r--  1 root root 37953 Nov 16 09:23 main.py
drwxr-xr-x  2 root root  4096 Nov 16 09:23 pretrained
-rw-r--r--  1 root root 16684 Nov 16 09:23 process_sql.py
-rw-r--r--  1 root root  4259 Nov 16 09:23 README.md
-rw-r--r-

In [ ]:
# Fix dependency issues first
!pip install -q jedi  # Fix ipython warning

# Install packages in the right order to avoid conflicts
print("Installing packages (this takes 2-3 minutes)...")

# Step 1: Core ML packages
!pip install -q torch torchvision torchaudio

# Step 2: NLP packages with compatible versions
!pip install -q 'transformers>=4.41.0,<5.0.0'
!pip install -q sentence-transformers
!pip install -q 'nltk>=3.9'  # Fix textblob conflict
!pip install -q openai

# Step 3: SQL and utility packages
!pip install -q 'sqlglot>=23.4'  # Use newer version
!pip install -q sqlparse
!pip install -q func-timeout
!pip install -q scikit-learn

# Step 4: Install vLLM last (takes longest)
print("\nInstalling vLLM (this takes 1-2 minutes)...")
!pip install -q vllm

print("\n✓ All packages installed successfully!")
print("\nInstalled versions:")
!pip list | grep -E 'torch|transformers|vllm|nltk|sqlglot'

Installing packages (this takes 2-3 minutes)...

Installing vLLM (this takes 1-2 minutes)...

✓ All packages installed successfully!

Installed versions:
nltk                                     3.9.2
sentence-transformers                    5.1.2
sqlglot                                  27.29.0
torch                                    2.8.0+cu126
torchao                                  0.10.0
torchaudio                               2.8.0+cu126
torchdata                                0.11.0
torchsummary                             1.5.1
torchtune                                0.6.1
torchvision                              0.23.0+cu126
transformers                             4.57.1
vllm                                     0.11.0


In [ ]:
import nltk
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

print("✓ NLTK data downloaded!")

✓ NLTK data downloaded!


In [ ]:
print("Fixing sentence_transformers compatibility...")

# Fix SentenceTransformer.py
with open('sentence_transformers/SentenceTransformer.py', 'r') as f:
    content = f.read()
content = content.replace(
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, cached_download',
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, hf_hub_download'
)
with open('sentence_transformers/SentenceTransformer.py', 'w') as f:
    f.write(content)

# Fix util.py
with open('sentence_transformers/util.py', 'r') as f:
    content = f.read()

content = content.replace(
    'from huggingface_hub import HfApi, hf_hub_url, cached_download, HfFolder',
    '''from huggingface_hub import HfApi, hf_hub_url, hf_hub_download, HfFolder
try:
    from huggingface_hub import cached_download
except ImportError:
    cached_download = None'''
)

with open('sentence_transformers/util.py', 'w') as f:
    f.write(content)

print("✓ Fixed sentence_transformers!")

Fixing sentence_transformers compatibility...
✓ Fixed sentence_transformers!


In [ ]:
from huggingface_hub import snapshot_download

print("Downloading BGE model (~1.3 GB)...")
print("This will take 2-3 minutes...\n")

model_path = snapshot_download(
    repo_id='BAAI/bge-large-en-v1.5',
    local_dir='./pretrained/bge-large-en-v1.5',
    local_dir_use_symlinks=False
)

print(f"\n✓ BGE model downloaded!")
!du -sh pretrained/bge-large-en-v1.5

This will take 2-3 minutes...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]


✓ BGE model downloaded!
3.8G	pretrained/bge-large-en-v1.5


In [ ]:
import os

os.makedirs('benchmark', exist_ok=True)

print("Downloading Spider dataset (~100 MB)...")
!wget -q --show-progress -O benchmark/spider.zip "https://drive.usercontent.google.com/download?id=1TqleXec_OykOYFREKKtschzY29dUcVAQ&export=download&confirm=t"

print("\nExtracting...")
!cd benchmark && unzip -q spider.zip && rm spider.zip

print("✓ Spider dataset ready!")
!du -sh benchmark/spider
!ls benchmark/spider/

benchmark/spider.zi 100%[===================>]  95.12M  67.1MB/s    in 1.4s    

Extracting...
✓ Spider dataset ready!
878M	benchmark/spider
database      dev.json	  tables.json	  train_others.json
dev_gold.sql  README.txt  train_gold.sql  train_spider.json


In [ ]:
from huggingface_hub import snapshot_download

print("Downloading TinyLlama-1.1B-Chat (~1 GB)...")
print("(Optimized for T4 GPU)\n")

model_path = snapshot_download(
    repo_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    local_dir='./models/TinyLlama-1.1B-Chat-v1.0',
    local_dir_use_symlinks=False
)

print("\n✓ LLM downloaded!")
!du -sh models/TinyLlama-1.1B-Chat-v1.0

(Optimized for T4 GPU)



Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

eval_results.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]


✓ LLM downloaded!
2.1G	models/TinyLlama-1.1B-Chat-v1.0


In [ ]:
try:
    vllm_process.terminate()
    vllm_process.wait(timeout=5)
    print("✓ Stopped old vLLM process")
except:
    pass

import subprocess, time, requests

print("Starting vLLM on port 8000 (avoiding port 8080)...\n")

vllm_log = open('vllm_server.log', 'w')
vllm_process = subprocess.Popen([
    'python', '-m', 'vllm.entrypoints.openai.api_server',
    '--model', './models/TinyLlama-1.1B-Chat-v1.0',
    '--host', '127.0.0.1',
    '--port', '8000',  # Changed from 8080 to 8000
    '--gpu-memory-utilization', '0.7',
    '--max-model-len', '2048',
], stdout=vllm_log, stderr=subprocess.STDOUT)

# Wait for server
print("Waiting for server to start...")
for i in range(60):
    try:
        response = requests.get('http://127.0.0.1:8000/v1/models', timeout=1)
        if response.status_code == 200:
            print(f"\n✓ vLLM server running on port 8000!")
            print(f"Models: {response.json()}")
            break
    except:
        if i % 5 == 0:
            print(f"  Waiting... {i+1}/60 sec")
        time.sleep(2)
else:
    print("\n⚠️ Still having issues. Checking logs:")
    !tail -20 vllm_server.log

✓ Stopped old vLLM process
Starting vLLM on port 8000 (avoiding port 8080)...

Waiting for server to start...
  Waiting... 1/60 sec
  Waiting... 6/60 sec
  Waiting... 11/60 sec
  Waiting... 16/60 sec
  Waiting... 21/60 sec
  Waiting... 26/60 sec

✓ vLLM server running on port 8000!
Models: {'object': 'list', 'data': [{'id': './models/TinyLlama-1.1B-Chat-v1.0', 'object': 'model', 'created': 1763285183, 'owned_by': 'vllm', 'root': './models/TinyLlama-1.1B-Chat-v1.0', 'parent': None, 'max_model_len': 2048, 'permission': [{'id': 'modelperm-66bd43cdf7d144859481f409e36dafb5', 'object': 'model_permission', 'created': 1763285183, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}]}]}


In [ ]:
# Update the LLM client to use port 8000
with open('main.py', 'r') as f:
    code = f.read()

# Change port from 8080 to 8000
code = code.replace("port=8080", "port=8000")
code = code.replace("LLMClient(host='127.0.0.1', port=8080)",
                   "LLMClient(host='127.0.0.1', port=8000)")

with open('main.py', 'w') as f:
    f.write(code)

print("✓ Updated main.py to use port 8000")

✓ Updated main.py to use port 8000


In [ ]:
print("Configuring main.py for Colab...\n")

with open('main.py', 'r') as f:
    code = f.read()

# Change to Spider dataset
code = code.replace("DB_NAME: str='bird'", "DB_NAME: str='spider'")

# Optimize for Colab T4
code = code.replace("FEW_SHOT_K = 8", "FEW_SHOT_K = 4")
code = code.replace("MAX_SCHEMAS = 30", "MAX_SCHEMAS = 20")
code = code.replace("MAX_COLUMNS = 200", "MAX_COLUMNS = 100")
code = code.replace("MAX_ITER = 5", "MAX_ITER = 3")
code = code.replace("max_workers=50", "max_workers=10")

# Test on 1 example first
code = code.replace("# data = data[:20]", "data = data[:1]  # Test 1 example")

with open('main.py', 'w') as f:
    f.write(code)

print("✓ Configuration complete!\n")
print("Settings:")
print("  Dataset: Spider")
print("  Few-shot examples: 4")
print("  Max schemas: 20")
print("  Max columns: 100")
print("  Max iterations: 3")
print("  Testing on: 1 example")

Configuring main.py for Colab...

✓ Configuration complete!

Settings:
  Dataset: Spider
  Few-shot examples: 4
  Max schemas: 20
  Max columns: 100
  Max iterations: 3
  Testing on: 1 example


In [ ]:
import nltk
import ssl

# Fix SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Download the new punkt_tab (required for NLTK 3.9+)
print("Downloading punkt_tab...")
nltk.download('punkt_tab', quiet=False)
nltk.download('averaged_perceptron_tagger_eng', quiet=False)

# Also download the old ones as backup
nltk.download('punkt', quiet=False)
nltk.download('averaged_perceptron_tagger', quiet=False)

print("\n✓ All NLTK data downloaded!")


✓ All NLTK data downloaded!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Kill old process
try:
    vllm_process.terminate()
    vllm_process.wait(timeout=5)
except:
    pass

import subprocess, time, requests

print("Starting vLLM with clean model name...\n")

vllm_log = open('vllm_server.log', 'w')
vllm_process = subprocess.Popen([
    'python', '-m', 'vllm.entrypoints.openai.api_server',
    '--model', './models/TinyLlama-1.1B-Chat-v1.0',
    '--served-model-name', 'TinyLlama-1.1B-Chat',  # ← This sets the clean name!
    '--host', '127.0.0.1',
    '--port', '8000',
    '--gpu-memory-utilization', '0.7',
    '--max-model-len', '2048',
], stdout=vllm_log, stderr=subprocess.STDOUT)

# Wait for server
print("Waiting for server...")
for i in range(60):
    try:
        response = requests.get('http://127.0.0.1:8000/v1/models', timeout=1)
        if response.status_code == 200:
            print(f"\n✓ vLLM server running!")
            print(f"Available models: {response.json()}")
            break
    except:
        if i % 5 == 0:
            print(f"  {i+1}/60 sec...")
        time.sleep(2)

Starting vLLM with clean model name...

Waiting for server...
  1/60 sec...
  6/60 sec...
  11/60 sec...
  16/60 sec...
  21/60 sec...
  26/60 sec...

✓ vLLM server running!
Available models: {'object': 'list', 'data': [{'id': 'TinyLlama-1.1B-Chat', 'object': 'model', 'created': 1763285243, 'owned_by': 'vllm', 'root': './models/TinyLlama-1.1B-Chat-v1.0', 'parent': None, 'max_model_len': 2048, 'permission': [{'id': 'modelperm-edf27fe367044d29a9db953f0e8ffea6', 'object': 'model_permission', 'created': 1763285243, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}]}]}


In [ ]:
# Create output directory
print("\n2. Creating output directory...")
import os
os.makedirs('output', exist_ok=True)
print("   ✓ Created")


2. Creating output directory...
   ✓ Created


In [ ]:
print("="*70)
print("RUNNING GEN-SQL ON 1 EXAMPLE")
print("="*70)
print("\nThis will take 30-60 seconds...\n")

!python main.py

print("\n" + "="*70)
print("✓ TEST COMPLETED!")
print("="*70)

RUNNING GEN-SQL ON 1 EXAMPLE

This will take 30-60 seconds...

Use Model: TinyLlama-1.1B-Chat
2025-11-16 09:27:32.006196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763285252.027605   37564 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763285252.034282   37564 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763285252.051290   37564 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763285252.051317   37564 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than

In [ ]:
import json
import glob

output_files = glob.glob('output/*.json')

if output_files:
    print(f"Found output: {output_files[0]}\n")

    with open(output_files[0], 'r') as f:
        results = json.load(f)

    if len(results) > 0 and results[0] is not None:
        r = results[0]

        print("="*70)
        print("📊 RESULT FOR FIRST EXAMPLE")
        print("="*70)

        print(f"\n📝 Question:")
        print(f"   {r['question']}")

        print(f"\n💾 Database:")
        print(f"   {r['db']}")

        print(f"\n✅ Ground Truth SQL:")
        print(f"   {r['output']}")

        print(f"\n🤖 Generated SQL:")
        print(f"   {r['infer']}")

        # Check if they match
        match = r['output'].strip().lower() == r['infer'].strip().lower()
        if match:
            print("\n🎉 EXACT MATCH! Perfect generation!")
        else:
            print("\n⚠️  Different from ground truth")
            print("   (May still be semantically correct)")

        print(f"\n📈 Statistics:")
        print(f"   Rounds/Iterations: {len(r['rounds'])}")
        print(f"   Stop reason: {r['stop_reason']}")
        print(f"   Total time: {r['time']:.2f} seconds")

        print("\n" + "="*70)
        print("🔄 ROUND-BY-ROUND BREAKDOWN")
        print("="*70)
        for i, rd in enumerate(r['rounds']):
            print(f"\nRound {i}:")
            sql_preview = rd['infer'][:70] + "..." if len(rd['infer']) > 70 else rd['infer']
            print(f"   SQL: {sql_preview}")
            print(f"   Time: {rd['time']:.2f}s")
            if 'usage' in rd:
                usage = rd['usage']
                print(f"   Tokens: prompt={usage.get('prompt_tokens', 0)}, completion={usage.get('completion_tokens', 0)}")

        print("\n" + "="*70)
        print("✅ GEN-SQL IS WORKING PERFECTLY!")
        print("="*70)
    else:
        print("⚠️ No valid results in output file")
else:
    print("❌ No output files found!")

Found output: output/TinyLlama-1.1B-Chat-spider-gtrr_column-S20C100-SS4CS6-L3-q_skltn-sql_skltn-sql_post_process-fs4-dev.json

📊 RESULT FOR FIRST EXAMPLE

📝 Question:
   How many singers do we have?

💾 Database:
   concert_singer

✅ Ground Truth SQL:
   SELECT count(*) FROM singer

🤖 Generated SQL:
   SELECT  COUNT(*) FROM singer_in_concert

⚠️  Different from ground truth
   (May still be semantically correct)

📈 Statistics:
   Rounds/Iterations: 2
   Stop reason: early_stopping
   Total time: 5.74 seconds

🔄 ROUND-BY-ROUND BREAKDOWN

Round 0:
   SQL: SELECT  count(*) FROM Singers
   Time: 3.37s
   Tokens: prompt=112, completion=7

Round 1:
   SQL: SELECT  COUNT(*) FROM singer_in_concert
   Time: 5.70s
   Tokens: prompt=563, completion=11

✅ GEN-SQL IS WORKING PERFECTLY!


In [ ]:
print("Configuring for 100 examples...\n")

with open('main.py', 'r') as f:
    code = f.read()
code = code.replace("data = data[:1]  # Test 1 example", "data = data[:100]  # Test 100 examples")
with open('main.py', 'w') as f:
    f.write(code)

print("✓ Ready to run on 100 examples")
print("\nRun the next cell to execute:")

Configuring for 100 examples...

✓ Ready to run on 100 examples

Run the next cell to execute:


In [ ]:
!python main.py

Use Model: TinyLlama-1.1B-Chat
2025-11-16 09:29:58.637782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763285398.658095   38205 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763285398.665032   38205 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763285398.687422   38205 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763285398.687449   38205 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763285398.687458   38205 computation_placer

In [ ]:
import glob
import json

# Find the output file
output_files = glob.glob('output/*.json')
print(f"Output file: {output_files[0]}\n")

# Quick stats
with open(output_files[0], 'r') as f:
    results = json.load(f)

results = [r for r in results if r is not None]
print(f"Total examples processed: {len(results)}")
print(f"Avg time per example: {sum(r['time'] for r in results)/len(results):.2f}s")
print(f"Total time: {sum(r['time'] for r in results):.2f}s\n")

# Show first few questions
print("Questions processed:")
for i, r in enumerate(results[:5]):
    print(f"  {i+1}. {r['question'][:60]}...")

Output file: output/TinyLlama-1.1B-Chat-spider-gtrr_column-S20C100-SS4CS6-L3-q_skltn-sql_skltn-sql_post_process-fs4-dev.json

Total examples processed: 41
Avg time per example: 10.84s
Total time: 444.53s

Questions processed:
  1. How many singers do we have?...
  2. What is the total number of singers?...
  3. What are the names, countries, and ages for every singer in ...
  4. What is the average, minimum, and maximum age of all singers...
  5. What is the average, minimum, and maximum age for all French...


In [ ]:
import json
import glob
import sqlite3
import os

print("="*70)
print("EM + EX EVALUATION")
print("="*70)

# Find results file (exclude reports)
results_files = [f for f in glob.glob('output/*.json')
                if '_report' not in f and '_errors' not in f]
results_file = results_files[0]
print(f"Results: {results_file}\n")

with open(results_file, 'r') as f:
    raw_results = json.load(f)

with open('benchmark/spider/dev.json', 'r') as f:
    dev_data = json.load(f)

# Filter out null entries and validate
results = []
null_count = 0
for i, r in enumerate(raw_results):
    if r is None:
        null_count += 1
    elif isinstance(r, dict) and 'infer' in r:
        results.append(r)
    else:
        print(f"  Unexpected type at {i}: {type(r)}")

print(f"Valid results: {len(results)}")
print(f"Null entries: {null_count}")
print(f"Total: {len(raw_results)}\n")

# Evaluation
total = len(results)
em_correct = 0
ex_correct = 0
execution_errors = 0

print(f"Evaluating {total} examples...\n")

for i, (result, gold) in enumerate(zip(results, dev_data[:total]), 1):
    # Exact Match
    pred_sql = result['infer'].strip().lower().replace('\n', ' ')
    gold_sql = gold['query'].strip().lower().replace('\n', ' ')

    if pred_sql == gold_sql:
        em_correct += 1

    # Execution Accuracy
    db_id = gold['db_id']
    db_path = f"benchmark/spider/database/{db_id}/{db_id}.sqlite"

    if os.path.exists(db_path):
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # Execute predicted
            cursor.execute(result['infer'])
            pred_results = sorted(cursor.fetchall())

            # Execute gold
            cursor.execute(gold['query'])
            gold_results = sorted(cursor.fetchall())

            conn.close()

            # Compare results
            if pred_results == gold_results:
                ex_correct += 1

        except Exception as e:
            execution_errors += 1
            if execution_errors <= 3:
                print(f"  ⚠️  Ex {i} error: {str(e)[:50]}...")

# Results
print("\n" + "="*70)
print("FINAL RESULTS")
print("="*70)
print(f"Examples evaluated:      {total}")
print(f"Null/Failed:             {null_count}")
print(f"\nExact Match (EM):        {em_correct}/{total} ({em_correct/total*100:.1f}%)")
print(f"Execution Accuracy (EX): {ex_correct}/{total} ({ex_correct/total*100:.1f}%)")
print(f"Execution errors:        {execution_errors}")

if ex_correct > em_correct:
    improvement = (ex_correct - em_correct) / total * 100
    print(f"\n✓ EX better than EM: +{improvement:.1f}%")
    print("  (Semantically correct despite syntax differences)")
elif ex_correct < em_correct:
    print(f"\n⚠️  EM > EX: {em_correct - ex_correct} queries have syntax match but execution issues")
else:
    print(f"\n= EM equals EX")

print("="*70)

# Show some examples
print("\n📋 EXAMPLE DETAILS (first 5):")
print("-"*70)
for i, (result, gold) in enumerate(zip(results[:5], dev_data[:5]), 1):
    print(f"\n{i}. {gold['question'][:55]}...")
    print(f"   Gold: {gold['query'][:60]}...")
    print(f"   Pred: {result['infer'][:60]}...")

    em = result['infer'].strip().lower() == gold['query'].strip().lower()
    print(f"   EM: {'✓' if em else '✗'}", end='')

    # Try execution
    db_path = f"benchmark/spider/database/{gold['db_id']}/{gold['db_id']}.sqlite"
    try:
        conn = sqlite3.connect(db_path)
        c = conn.cursor()
        c.execute(result['infer'])
        pr = sorted(c.fetchall())
        c.execute(gold['query'])
        gr = sorted(c.fetchall())
        conn.close()
        ex_match = pr == gr
        print(f", EX: {'✓' if ex_match else '✗'}")
    except:
        print(f", EX: Error")

EM + EX EVALUATION
Results: output/TinyLlama-1.1B-Chat-spider-gtrr_column-S20C100-SS4CS6-L3-q_skltn-sql_skltn-sql_post_process-fs4-dev.json

Valid results: 41
Null entries: 59
Total: 100

Evaluating 41 examples...

  ⚠️  Ex 23 error: no such table: Pets...
  ⚠️  Ex 24 error: no such table: Pets...
  ⚠️  Ex 25 error: no such table: Pets...

FINAL RESULTS
Examples evaluated:      41
Null/Failed:             59

Exact Match (EM):        0/41 (0.0%)
Execution Accuracy (EX): 0/41 (0.0%)
Execution errors:        19

= EM equals EX

📋 EXAMPLE DETAILS (first 5):
----------------------------------------------------------------------

1. How many singers do we have?...
   Gold: SELECT count(*) FROM singer...
   Pred: SELECT  COUNT(*) FROM singer_in_concert...
   EM: ✗, EX: ✗

2. What is the total number of singers?...
   Gold: SELECT count(*) FROM singer...
   Pred: SELECT  COUNT(*) FROM singer_in_concert...
   EM: ✗, EX: ✗

3. Show name, country, age for all singers ordered by age ...
   Gold: 